## Winning Jeopardy!

![Image](https://www.nerdwallet.com/assets/blog/wp-content/uploads/2015/08/Winning-money.png)

### Introduction

[Jeopardy](https://en.wikipedia.org/wiki/Jeopardy!) is a popular TV show in the US where participants answer questions to win money. In this project we'll try to find a way to become the winner and get as much money as possible.

We'll be working with the data set from [reddit](https://www.reddit.com/r/datasets/comments/1uyd0t/200000_jeopardy_questions_in_a_json_file/). It contains **216930** questions and other info. Let's take a look.

In [1]:
import pandas as pd

jeopardy = pd.read_csv('jeopardy.csv', parse_dates=[' Air Date'])
jeopardy.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [2]:
jeopardy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216930 entries, 0 to 216929
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Show Number  216930 non-null  int64         
 1    Air Date    216930 non-null  datetime64[ns]
 2    Round       216930 non-null  object        
 3    Category    216930 non-null  object        
 4    Value       216930 non-null  object        
 5    Question    216930 non-null  object        
 6    Answer      216928 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 11.6+ MB


### Cleaning

Before we start let's clean some mess. Several columns have spaces in front. Let's fix it.

In [3]:
cols_to_fix = jeopardy.columns
print('Old \n', cols_to_fix)

cols_to_fix = cols_to_fix.str.replace(r'^ ', '', regex=True)
print('Fixed \n',cols_to_fix)

jeopardy.columns = cols_to_fix

Old 
 Index(['Show Number', ' Air Date', ' Round', ' Category', ' Value',
       ' Question', ' Answer'],
      dtype='object')
Fixed 
 Index(['Show Number', 'Air Date', 'Round', 'Category', 'Value', 'Question',
       'Answer'],
      dtype='object')


Also let's normalize strings in the `Question` and `Answer` columns. By that I mean:
* Everyword should be in lowercase
* No punctuation

The `re` library will help us with that. But we are missing two values in the `Answer` column, so let's remove these rows first.

In [4]:
import re

jeopardy.dropna(subset=['Answer'], inplace=True)

def clean_qa(string):
    '''
    Take a string and return it in lowercase and
    without any punctuation 
    '''
    string = string.lower()
    string = re.sub(r'[^\w\s]', '', string)
    return string

jeopardy['clean_question'] = jeopardy['Question'].apply(clean_qa)
jeopardy['clean_answer'] = jeopardy['Answer'].apply(clean_qa)
jeopardy[['clean_question', 'clean_answer']].head()

,clean_question,clean_answer
0,for the last 8 years of his life galileo was u...,copernicus
1,no 2 1912 olympian football star at carlisle i...,jim thorpe
2,the city of yuma in this state has a record av...,arizona
3,in 1963 live on the art linkletter show this c...,mcdonalds
4,signer of the dec of indep framer of the const...,john adams


We also can turn `Value` column to the numeric to allow us to manipulate it easier. But we must remove **\$** and **,** signs first.

Some values are missing, we'll replace them with 0.

In [5]:
def clean_value(string):
    '''
    Take a string, remove $ and , signs and try to covert it to int
    
    If can't convert return 0
    '''
    string = re.sub(r'[$,]', '', string)
    
    try:
        string = int(string)
    except:
        string = 0
        
    return string

jeopardy['clean_value'] = jeopardy['Value'].apply(clean_value)
jeopardy['clean_value'].head()

0    200
1    200
2    200
3    200
4    200
Name: clean_value, dtype: int64

### Analysis

In order to figure out whether to study past questions, study general knowledge, or not study it all, it would be helpful to figure out two things:
* How often the answer can be used for a question
* How often questions are repeated

#### The answer from the question

We can answer the first question by seeing how many times words in the answer also occur in the question. Also we'll remove `The` from count due to it's commonly found in answers and questions, but doesn't have any meaningful use.

Let's do this.

In [6]:
def count_match(row):
    '''
    Take a row from dataframe and count how many
    times word from answer is in also in question
    
    Return relative result
    '''
    split_question = row['clean_question'].split()
    split_answer = row['clean_answer'].split()
    
    match_count = 0
    
    #Remove `the` from answer if it's in there
    if 'the' in split_answer:
        split_answer.remove('the')
        
    if split_answer:
        for word in split_answer:
            if word in split_question:
                match_count += 1
                
        return match_count / len(split_answer)
        
    else:
        return 0

jeopardy['answer_in_question'] = jeopardy.apply(count_match, axis=1)
print(jeopardy['answer_in_question'].mean())
jeopardy['answer_in_question'].head()

0.05792123724515945


0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: answer_in_question, dtype: float64

As we can see only **5.79%** words from the answer are also in the question on average. It means that we can't hope to find the right answer just by hearing the question. Probably we should study.

#### Repeated questions

So we want to investigate how often new questions are repeats of older ones. For that purpose we'll:
* go from the olderest questions to the most recent
* look for any word that has 6 characters at least
* add this word to the kind of vocabulary
* count how many times word's arleady occurred

In [7]:
#Sord df by date first
jeopardy.sort_values(by=['Air Date'], inplace=True)

#Prepare the dictionary
terms_used = {}

def overlap_count(question):
    '''
    Take a question, remove all short words (< 6 chars)
    and make up a dictionary of unique words
    
    Return relative overlap number
    '''
    split_question = question.split()
    
    #Remove short words (< 6 char)
    split_question = [word for word in split_question if len(word)>5]
    
    #Count the occurrence of word
    match_count = 0
    for word in split_question:
        if word in terms_used.keys():
            match_count += 1
    
    #Fill the dictionary
    for word in split_question:
        if word in terms_used:
            value = terms_used[word]
            terms_used[word] = match_count + value
        else:
            terms_used[word] = match_count
    
    if len(split_question) != 0:
        overlap = match_count / len(split_question)
        return overlap
    else:
        return match_count
        
jeopardy['question_overlap'] = jeopardy['clean_question'].apply(overlap_count)

print(jeopardy['question_overlap'].mean())
jeopardy['question_overlap'].head()

0.8721734034757384


84523    0.0
84565    0.0
84566    0.0
84567    0.0
84568    0.0
Name: question_overlap, dtype: float64

There is about **87%** overlap between terms in new questions and old questions. So we can use old questions to prepare to the game.

But this set include about **10%** of all jeopardy question. And we looked only on separate words but not phrases. Using only overlaping question isn't ther ultimate solution.

#### Question value

We could deside to pay attention to the high value question only. Given that we must separate all questions into two categories, for example:
* low value - any question worth less than \$800
* high value - any question worth \$800 or more

Then we can find which terms correspond to the high-value questions.

In [8]:
def high_low(value):
    '''
    Put a label on the question depends on it's value
    '''
    if value < 800:
        return 0
    else:
        return 1
    
jeopardy['high_value'] = jeopardy['clean_value'].apply(high_low)
jeopardy['high_value'].head()

84523    0
84565    1
84566    1
84567    1
84568    1
Name: high_value, dtype: int64

Now we can start to count word occurrences in each category.

In [9]:
def high_low_count(word):
    '''
    Take a word and look for overlaps in each category
    
    Return both high and low counts
    '''
    high_count = 0
    low_count = 0
    
    for index, row in jeopardy.iterrows():
        split_question = row['clean_question'].split()
        
        if word in split_question:
            if row['high_value'] == 1:
                high_count += 1
            else:
                low_count += 1
                
    return high_count, low_count

import random

#Take 10 random terms
random.seed(0)
comparison_terms = random.sample(tuple(terms_used), k=10)

observed_expected = []
for word in comparison_terms:
    result = high_low_count(word)
    observed_expected.append(result)
    
observed_expected

[(0, 1),
 (1, 0),
 (0, 1),
 (0, 1),
 (5, 8),
 (1, 0),
 (2, 0),
 (1, 0),
 (0, 1),
 (1, 0)]

In [10]:
comparison_terms

['hrefhttpwwwjarchivecommedia20010924_dj_27jpg',
 'hrefhttpwwwjarchivecommedia20111216_dj_17wmvjimmy',
 'dunoon',
 'hrefhttpwwwjarchivecommedia19861016_j_07jpg',
 'saturated',
 'hrefhttpwwwjarchivecommedia20051207_j_04jpg',
 'theatera',
 'bonerz',
 'luyendyk',
 'target_blankconditionsa']

In [11]:
def high_low_count(row):
    '''
    Take a question, remove all short words (< 6 chars)
    and make up a dictionary of unique words according
    question value
    
    Return filled dictionary
    '''
    split_question = row['clean_question'].split()
    
    #Remove short words (< 6 char)
    split_question = [word for word in split_question if len(word)>5]
    
    #Fill the dictionary
    if row['high_value'] == 1:
        for word in split_question:
            terms_high_low[word][0] +=1
    else:
        for word in split_question:
            terms_high_low[word][1] +=1
    
    return terms_high_low

In [16]:
aa = {w:[0,0] for w in terms_used.keys()}
aa

{'mentioned': [0, 0],
 'according': [0, 0],
 'timothy': [0, 0],
 'experimenter': [0, 0],
 'herbert': [0, 0],
 'taught': [0, 0],
 'viewers': [0, 0],
 'science': [0, 0],
 'building': [0, 0],
 'shaken': [0, 0],
 'november': [0, 0],
 'leaped': [0, 0],
 'shouting': [0, 0],
 'semper': [0, 0],
 'tyrannis': [0, 0],
 'president': [0, 0],
 'before': [0, 0],
 'stepping': [0, 0],
 'office': [0, 0],
 'monday': [0, 0],
 'january': [0, 0],
 'starting': [0, 0],
 'american': [0, 0],
 'shorter': [0, 0],
 'mississippi': [0, 0],
 'jewish': [0, 0],
 'filled': [0, 0],
 'cheese': [0, 0],
 '4letter': [0, 0],
 'spangled': [0, 0],
 'banner': [0, 0],
 'husbands': [0, 0],
 'ermine': [0, 0],
 'theyre': [0, 0],
 'actually': [0, 0],
 'buying': [0, 0],
 'hrefhttpwwwjarchivecommedia19840910_j_14mp3thisa': [0, 0],
 'famous': [0, 0],
 'saturday': [0, 0],
 'alumni': [0, 0],
 'trading': [0, 0],
 'places': [0, 0],
 'british': [0, 0],
 'variety': [0, 0],
 'called': [0, 0],
 'bangers': [0, 0],
 'mexican': [0, 0],
 'chorizo':

In [14]:
#Prepare the dictionary
terms_high_low = {w:[0,0] for w in terms_used.keys()}

freq_dict = jeopardy.apply(high_low_count, axis=1)
freq_dict

84523     {'mentioned': [88, 122], 'according': [165, 40...
84565     {'mentioned': [88, 122], 'according': [165, 40...
84566     {'mentioned': [88, 122], 'according': [165, 40...
84567     {'mentioned': [88, 122], 'according': [165, 40...
84568     {'mentioned': [88, 122], 'according': [165, 40...
                                ...                        
105945    {'mentioned': [88, 122], 'according': [165, 40...
105946    {'mentioned': [88, 122], 'according': [165, 40...
105947    {'mentioned': [88, 122], 'according': [165, 40...
105939    {'mentioned': [88, 122], 'according': [165, 40...
105941    {'mentioned': [88, 122], 'according': [165, 40...
Length: 216928, dtype: object

In [17]:
freq_dict[84523]

{'mentioned': [88, 122],
 'according': [165, 409],
 'timothy': [19, 21],
 'experimenter': [3, 0],
 'herbert': [36, 47],
 'taught': [91, 91],
 'viewers': [18, 37],
 'science': [170, 166],
 'building': [275, 410],
 'shaken': [8, 8],
 'november': [148, 250],
 'leaped': [4, 8],
 'shouting': [3, 9],
 'semper': [9, 12],
 'tyrannis': [2, 4],
 'president': [1254, 1756],
 'before': [1225, 1684],
 'stepping': [8, 14],
 'office': [248, 367],
 'monday': [38, 75],
 'january': [168, 331],
 'starting': [85, 93],
 'american': [1354, 1483],
 'shorter': [27, 34],
 'mississippi': [125, 146],
 'jewish': [131, 144],
 'filled': [61, 95],
 'cheese': [156, 268],
 '4letter': [147, 148],
 'spangled': [3, 3],
 'banner': [27, 44],
 'husbands': [36, 31],
 'ermine': [1, 5],
 'theyre': [290, 462],
 'actually': [205, 225],
 'buying': [30, 47],
 'hrefhttpwwwjarchivecommedia19840910_j_14mp3thisa': [1, 0],
 'famous': [1053, 1444],
 'saturday': [52, 86],
 'alumni': [14, 8],
 'trading': [50, 34],
 'places': [67, 89],
 'br

In [ ]:
def high_low_count(question):
    '''
    Take a word and look for overlaps in each category
    
    Return both high and low counts
    '''
    high_count = 0
    low_count = 0
    
    split_question = question.split()
        
    if word in split_question:
        if row['high_value'] == 1:
            high_count += 1
        else:
            low_count += 1
                
    return high_count, low_count

import random

#Take 10 random terms
random.seed(0)
comparison_terms = random.sample(tuple(terms_used), k=10)

observed_expected = []
for word in comparison_terms:
    result = jeopardy['clean_question'].apply(high_low_count(word=word))
    observed_expected.append(result)
    
observed_expected

Now we've found the observed counts for the chi-squared test.

#### Chi-Squared test

We are almost reasy for chi-squared test, we just need the expected values. We can get them by using total terms proportions and overall number of high-value and low-value questions.

In [ ]:
from scipy.stats.mstats import chisquare

#Numbers of question in each category
total_high = jeopardy['high_value'].sum()
total_low = len(jeopardy) - total_high

chi_squared = []
for pair in observed_expected:
    pair_sum = pair[0] + pair[1]
    total_prop = pair_sum / len(jeopardy)
    
    #Expected values
    high_exp = total_high * total_prop
    low_exp = total_low * total_prop
    
    chi_result = chisquare(pair, [high_exp, low_exp])
    chi_squared.append(chi_result)
    
chi_squared

None of the terms had a significant difference in usage between high value and low value rows. Also most of the frequencies were low, so the chi-squared test isn't as valid.

#### High frequency chi-Squared

It would be better to run this test with only terms that have higher frequencies.

In [ ]:
top_used = pd.Series(terms_used).sort_values(ascending=False)
top_used.head(10)

Let's apply chi-squared test for top 50 words.

In [ ]:
top_50 = top_used.iloc[:50]

chi_squared_top = []
for obs in top_50:
def chi_top(observed)    
    
]
    total_prop = obs / len(jeopardy)
    
    #Expected values
    high_exp = total_high * total_prop
    low_exp = total_low * total_prop
    
    chi_result = chisquare(pair, [high_exp, low_exp])
    chi_squared.append(chi_result)
    
chi_squared